# Analize e processamento do TK #9 do `scieloorg/doi_request`

Para mais informações sobre o problema e motivações olhar o link: 
https://github.com/scieloorg/doi_request/issues/9

In [ ]:
import re
import pandas as pd
import numpy as np

Arquivo CSV gerado com os dados dos documentos da coleção SciELO Brasil que não possuem registro no Crossref.

In [ ]:
doi_files = pd.read_csv('./SciELO_Brazil_DOI.csv', delimiter=";", low_memory=False)
doi_files.head()

In [ ]:
doi_files.shape

#### Testar se o doi ainda não esta registrado no `https://www.doi.org`

Para testar se os dois ainda estao indisponiveis, foi construido o script `generate_doi_not_found.py` que consulta toda a lista dos dois no proprio site do `www.doi.org`

Esse script deve ser executado num virtualenv de `python>=3.7`e que tenha cido instalados as dependencias atraves do compando `pip install -r requirements.generate_doi_not_found.txt`

```shell
$ python generate_doi_not_found.py
```

Ao termido desse processamento teremos o arquivo `df_doi_not_found.csv` e podemos continuar a analize.
Nessa etapa devemos pesquizar no *SciELO - DOI Manager* se esse PID ja foi processados ou não, para isso foi construido o script `consult_doi_request_and_extract_data.py` que o ira consultar o banco de dados da aplicação e constroi os arquivos `df_doi_not_processed.csv`, `df_doi_processed.csv` com os dados de dois processado e nao processado. 

```shell
$ python consult_doi_request_and_extract_data.py \
    --host HOSTDB \
    --port 5432 \
    --user doi_user 
    --password XXXXXXX 
    --database doi_manager
```

In [ ]:
# Carregando dados dos arquivos gerados para facilitar processo
df_doi_not_processed = pd.read_csv("./df_doi_not_processed.csv")
df_doi_processed = pd.read_csv("./df_doi_processed.csv")

In [ ]:
print("DOIs nunca processados", len(df_doi_not_processed))
df_doi_not_processed.head()

In [ ]:
print("DOIs ja processados", len(df_doi_processed))
df_doi_processed.head()

## Analizando erros do DOI não pocessados

 Esses casos nessecitam ser Processados adicionados pelo SciELO - DOI Manager

In [ ]:
# Lista dos PIDs para serem Processados
print("\n".join(df_doi_not_processed.pid.to_list()))

## Analizando erros do DOI ja processados

Foi extraido os dados de `Situação de depósito` e `XML de resultado do depósito` diretamente do banco de dados do PostgreSQL atraves do script utilizado anteriormente.

In [ ]:
# tratando cada caso de retorno
gb = df_doi_processed.groupby("feedback_status")
data_errors = {
    x: gb.get_group(x) for x in gb.groups
}

for k,v in data_errors.items():
    print("status:", k, "\tTotal", v.journal.count())

### Caso de  statos com `failure`

In [ ]:
status_failure = data_errors["failure"]
print("Total de item com failure", status_failure.journal.count())

In [ ]:
# Regex para tratar os retornos dos xml do CrossRef
regex = r"<msg>(.*)<\/msg>"
result = []

for item in status_failure.itertuples():
    group = re.findall(regex, item.feedback_xml, re.MULTILINE)
    if group:
        msg_feedback = group[0]
    else:
        msg_feedback = ""
    result.append(
        [item.journal, msg_feedback]
    )

df_result = pd.DataFrame(result, columns=["journal", "msg_feedback"])
g_df_result = df_result.groupby(["msg_feedback", "journal"]).size().to_frame('size')
g_df_result.sort_values(by="size", ascending=False)



### Caso de  statos com `success`

Esse item deve ser analidados em separado pois aparentemente eles foram processados com sucesso pelo CrossRef, mais seu DOI acusa erro no relatorio

In [ ]:
status_success = data_errors["success"]
print("Total de item com success", status_success.journal.count())

In [ ]:
status_success.head()

In [ ]:
import urllib.request
exclude_pid = []
for row in status_success.itertuples():
    req = urllib.request.Request("https://www.doi.org/{0}".format(row.doi))
    try: 
        urllib.request.urlopen(req)
    except urllib.error.URLError as e:
        print(e.reason)
    else:
        exclude_pid.append(row.pid)

Esses casos os PIDs abaixo deve ser excluidos da lista de erros, pois eles existem e estao validos e registrados

In [ ]:
# Lista dos PIDs para excluir do relatorio
print("Total de PIDS: ", len(exclude_pid))

print("\n".join(exclude_pid))

### Caso de  statos com `error` e `waiting`

In [ ]:
status_error = data_errors["error"]
status_waiting = data_errors["waiting"]

print("Total de item com waiting", status_waiting.journal.count())
print("Total de item com error", status_error.journal.count())

In [ ]:
status_error.head()

Esses casos nessecitam ser reprocessados pelo SciELO - DOI Manager

In [ ]:
status_waiting.head()

Esses casos nessecitam ser reprocessados pelo SciELO - DOI Manager

In [ ]:
# Lista dos PIDs para serem Reprocessados
print("\nstatus_waiting\n")
print("\n".join(status_waiting.pid.to_list()))
print("\nstatus_error\n")
print("\n".join(status_error.pid.to_list()))

### Caso de  statos com `notapplicable` 

In [ ]:
status_notapplicable = data_errors["notapplicable"]

print("Total de item com status_notapplicable", status_notapplicable.journal.count())

In [ ]:
status_notapplicable.head()

Esse caso deve se verificar o *Prefix* utilizado para registro dos DOI, pois aparetentemente ouve erro de digitação necesses dados. e não há como reprocesser sem a correçao do valor do prefixo.

In [ ]:
print("\n".join(status_notapplicable.doi.to_list()))